In [2]:
import os

In [4]:
print(os.getcwd())
os.chdir("../")
print(os.getcwd())

'c:\\Users\\A2Z\\Desktop\\MLOPs\\cicd-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path 
    unzip_dir: Path

In [10]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
            self, 
            config_file_path = CONFIG_FILE_PATH, 
            params_file_path = PARAMS_FILE_PATH, 
            schema_file_path = SCHEMA_FILE_PATH
    ):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir, 
            source_url = config.source_url, 
            local_data_file = config.local_data_file, 
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [17]:
import zipfile
import urllib.request as request
import os
from mlProject import logger
from mlProject.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    def donwlaod_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with info {header}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as ref_zip:
            ref_zip.extractall(self.config.unzip_dir)

        

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.donwlaod_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e